In [5]:
#!pip install pandas google-cloud-bigquery ipython-sql
#!pip install db-dtypes
!pip install google-cloud-bigquery-storage


In [18]:
import os
from google.cloud import bigquery
import pandas as pd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cmpe180b-project-f6eb9de16408.json"



In [19]:
client = bigquery.Client()

# Example query
query = """
SELECT state_name, COUNT(*) as fatalities
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2015`
GROUP BY state_name
ORDER BY fatalities DESC
LIMIT 5
"""

df = client.query(query).to_dataframe()
df.head()


,state_name,fatalities
0,Texas,3189
1,California,3067
2,Florida,2698
3,Georgia,1328
4,North Carolina,1273


## Fetch all table names

In [28]:

# Initialize the client
client = bigquery.Client()

# Replace with your project and dataset ID
project_id = "bigquery-public-data"
dataset_id = "nhtsa_traffic_fatalities"

# Get a reference to the dataset
dataset_ref = client.dataset(dataset_id, project=project_id)

# List all tables in the dataset
tables = client.list_tables(dataset_ref)

# Print all table names
#for table in tables:
#    print(table.table_id)

table_list = [table.table_id for table in tables]
df_tables = pd.DataFrame(table_list, columns=["table_name"])
# Option 2: Using pd.option_context (temporary change)

# We only want the first 103 tables, because the rest are repeated
df_tables = df_tables[:103]
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 0):
    print(df_tables)



          table_name
0      accident_2015
1      accident_2016
2      accident_2017
3      accident_2018
4      accident_2019
5      accident_2020
6        cevent_2015
7        cevent_2016
8        cevent_2017
9        cevent_2018
10       cevent_2019
11       cevent_2020
12       damage_2015
13       damage_2016
14       damage_2017
15       damage_2018
16       damage_2019
17       damage_2020
18     distract_2015
19     distract_2016
20     distract_2017
21     distract_2018
22     distract_2019
23     distract_2020
24     drimpair_2015
25     drimpair_2016
26     drimpair_2017
27     drimpair_2018
28     drimpair_2019
29     drimpair_2020
30       factor_2015
31       factor_2016
32       factor_2017
33       factor_2018
34       factor_2019
35       factor_2020
36     maneuver_2015
37     maneuver_2016
38     maneuver_2017
39     maneuver_2018
40     maneuver_2019
41     maneuver_2020
42      nmcrash_2015
43      nmcrash_2016
44      nmcrash_2017
45      nmcrash_2018
46      nmcra

## Fetch all attributes of accident_2015

In [21]:

# Initialize client
client = bigquery.Client()

# Full table ID in the format: project.dataset.table
table_id = "bigquery-public-data.nhtsa_traffic_fatalities.accident_2015"

# Get table metadata
table = client.get_table(table_id)

# Print column names and data types
for schema_field in table.schema:
    print(f"{schema_field.name}: {schema_field.field_type}")


state_number: INTEGER
state_name: STRING
consecutive_number: INTEGER
number_of_vehicle_forms_submitted_all: INTEGER
number_of_motor_vehicles_in_transport_mvit: INTEGER
number_of_parked_working_vehicles: INTEGER
number_of_forms_submitted_for_persons_not_in_motor_vehicles: INTEGER
number_of_persons_not_in_motor_vehicles_in_transport_mvit: INTEGER
number_of_persons_in_motor_vehicles_in_transport_mvit: INTEGER
number_of_forms_submitted_for_persons_in_motor_vehicles: INTEGER
county: INTEGER
city: INTEGER
day_of_crash: INTEGER
month_of_crash: INTEGER
year_of_crash: INTEGER
day_of_week: INTEGER
hour_of_crash: INTEGER
minute_of_crash: INTEGER
national_highway_system: INTEGER
land_use: INTEGER
land_use_name: STRING
functional_system: INTEGER
functional_system_name: STRING
ownership: INTEGER
ownership_name: STRING
route_signing: INTEGER
route_signing_name: STRING
trafficway_identifier: STRING
trafficway_identifier_2: STRING
milepoint: INTEGER
latitude: FLOAT
longitude: FLOAT
special_jurisdiction

In [31]:
# Initialize client
client = bigquery.Client()

# Store all schema details
schema_rows = []

# Loop through all 2015 tables in df_tables
for table_name in df_tables["table_name"]:
    full_table_id = f"bigquery-public-data.nhtsa_traffic_fatalities.{table_name}"
    
    try:
        table = client.get_table(full_table_id)
        for field in table.schema:
            schema_rows.append({
                "table_name": table_name,
                "column_name": field.name,
                "data_type": field.field_type
            })
    except Exception as e:
        print(f"Failed to get schema for {table_name}: {e}")

# Create a DataFrame with all schema info
df_schema_all = pd.DataFrame(schema_rows)

# Preview the result
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 0):
    # print(df_schema_all)
df_schema_all.to_csv("nhtsa_2015_table_schemas.csv", index=False)
